# Exemplo de um código em Python para extrair dados de emails do Google Workspace

Primeiro instale a biblioteca do google para o python:

In [22]:
pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client



[notice] A new release of pip available: 22.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Código para listar E-mails

Impotação das bibliotecas necessárias

In [17]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

Caminho do JSON da service account

In [8]:
SERVICE_ACCOUNT_FILE = "seu_arquivo.json"

Scopes necessários (garanta que já estão aprovados)

In [9]:
SCOPES = [
    "https://www.googleapis.com/auth/admin.directory.user.readonly",
    "https://www.googleapis.com/auth/gmail.readonly"
]

E-mail do administrador do Google Workspace e autenticação do serviço

In [18]:
ADMIN_EMAIL = "admin@empresa.com"
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
).with_subject(ADMIN_EMAIL)

FileNotFoundError: [Errno 2] No such file or directory: 'seu_arquivo.json'

Criação dos serviços

In [ ]:
admin_service = build("admin", "directory_v1", credentials=credentials)
gmail_service = build("gmail", "v1", credentials=credentials)

Listar usuários da empresa

In [15]:

def list_all_users():
    """Lista todos os usuários da empresa"""
    users = []
    request = service.users().list(customer="my_customer", maxResults=100, orderBy="email")

    while request is not None:
        response = request.execute()
        users.extend(response.get("users", []))
        request = service.users().list_next(previous_request=request, previous_response=response)

    for user in users:
        print(f"Nome: {user['name']['fullName']}")
        print(f"E-mail: {user['primaryEmail']}")
        print(f"Status: {user['suspended']}")
        print("-" * 50)

Obter e-mail dos usuários

In [19]:
def get_all_users():
    """Lista todos os usuários do Google Workspace"""
    users = []
    request = admin_service.users().list(customer="my_customer", maxResults=100, orderBy="email")

    while request is not None:
        response = request.execute()
        users.extend(response.get("users", []))
        request = admin_service.users().list_next(previous_request=request, previous_response=response)

    return [user["primaryEmail"] for user in users]

Obtem o email para cada usuário

In [20]:

def get_emails_for_user(user_email):
    """Lista os últimos 5 e-mails de um usuário"""
    try:
        results = gmail_service.users().messages().list(userId=user_email, maxResults=5).execute()
        messages = results.get("messages", [])

        for msg in messages:
            message = gmail_service.users().messages().get(userId=user_email, id=msg["id"]).execute()
            headers = message["payload"]["headers"]

            sender = recipient = date = subject = None
            for header in headers:
                if header["name"] == "Date":
                    date = header["value"]
                if header["name"] == "From":
                    sender = header["value"]
                if header["name"] == "To":
                    recipient = header["value"]
                if header["name"] == "Subject":
                    subject = header["value"]

            print(f"Usuário: {user_email}")
            print(f"De: {sender}")
            print(f"Para: {recipient}")
            print(f"Data: {date}")
            print(f"Assunto: {subject}")
            print("-" * 50)

    except Exception as e:
        print(f"Erro ao buscar e-mails de {user_email}: {e}")

Listar todos os usuários e buscar os e-mails de cada um

In [21]:
all_users = get_all_users()
for user in all_users:
    get_emails_for_user(user)

NameError: name 'admin_service' is not defined